In [0]:
'''
#Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import SimpleRNN
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [0]:
path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

606208/600901 [==============================] - 0s 0us/step
corpus length: 600893


In [0]:
text[:1000]

'preface\n\n\nsupposing that truth is a woman--what then? is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to truth, have been unskilled and unseemly methods for\nwinning a woman? certainly she has never allowed herself to be won; and\nat present every kind of dogma stands with sad and discouraged mien--if,\nindeed, it stands at all! for there are scoffers who maintain that it\nhas fallen, that all dogma lies on the ground--nay more, that it is at\nits last gasp. but to speak seriously, there are good grounds for hoping\nthat all dogmatizing in philosophy, whatever solemn, whatever conclusive\nand decided airs it has assumed, may have been only a noble puerilism\nand tyronism; and probably the time is at hand when it will be once\nand again understood what has actually sufficed for the basis of such\

In [0]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))


total chars: 39
nb sequences: 371785


In [0]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


Vectorization...


In [0]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


W0803 12:51:32.049930 140000625973120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0803 12:51:32.085875 140000625973120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0803 12:51:32.093389 140000625973120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



Build model...


W0803 12:51:32.384724 140000625973120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0803 12:51:32.394182 140000625973120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [0]:
a = np.array([0.1,0.2,0.3,0.4])
preds = np.log(a) / 1
exp_preds = np.exp(preds)
preds = exp_preds / np.sum(exp_preds)
np.random.multinomial(1000, preds, 1)

array([[ 97, 213, 324, 366]])

In [0]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature # temperature 가 1일 가까울 수록 원래 확률대로 나옴
    # 반면 0에 가까울수록 dominating 할 가능성이 커짐 (각 확률을 1/temparature 승)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


In [0]:

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=100,
          callbacks=[print_callback])

In [0]:
# build the model: a single RNN

print('Build model...')
model3 = Sequential()
model3.add(SimpleRNN(128, input_shape=(maxlen, len(chars))))
model3.add(Dense(len(chars), activation='softmax'))

optimizer3 = RMSprop(lr=0.01)
model3.compile(loss='categorical_crossentropy', optimizer=optimizer3)

model3.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Build model...
Epoch 1/60
200285/200285 [==============================] - 52s 261us/step - loss: 2.7742

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: " himself--and
what is really his answer?"
 himself--and
what is really his answer? it is a princerable to the standing of the belief in the responsible of the spiritual of the standard of the process of the subble of the sense of the spiritude of the sense of the other in the sense of the soul, and the present of the most process and the subjection of the sentiment of the processed to the presented and the possible of the sentiment of the presence of the same action of the poss
----- diversity: 0.5
----- Generating with seed: " himself--and
what is really his answer?"
 himself--and
what is really his answer? it is a soul them contrary, the obtains to the saticism, and and in the portality of the alited to used the end, the fact that the portality of called the strives of the present with somethin

G:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


efories? precisely rernsormentanted to new "howeffounded loikesm of
dlepe uponise,
griatish hop
in go abableness is endiscintes. letteved, whichs soxmable mere afolme brings, when sacrive it, hence
is not penourisoty, mide
hibile until an ers, not
a menstend involrerce and woyings, is being halbitgest bour malies her head freepheible. a sa
Epoch 2/60
200285/200285 [==============================] - 51s 253us/step - loss: 2.6672

----- Generating text after Epoch: 1
----- diversity: 0.2
----- Generating with seed: "tates. if a man is brought to an extraor"
tates. if a man is brought to an extraordinary of the present and intention, the content to the sentiment which the same order of the sense of the conscience of the sentiment of the possible of the general of the most standards of the present and a princeral the processed to the possible of the present of the art of the sentiment of the present of the principle of the same only and the spect of the spiritude of the possible of the sub

and a present to the present and intention, and the subject of the sense of the result of the sense of the same as the present and intention, and and as the content of the sense of the sense of the sense of the spirits of the sense of the processed to the standing of the sense of the most extentary of the possess, the presented of the other of the spect of the spirits of the same the and the sacrific
----- diversity: 0.5
----- Generating with seed: "time is at hand when it will be once
and"
time is at hand when it will be once
and to be
will to the possible and soul standards of his depended to such a sacrifices of belongs and the standing
of love of the present silence of the content of the condition of the case of the spirity and the religious subble of man the prinception of the
responsible which whether the twog or and the condition of the process and possible and the world and the spect of which he fain the being. in 
----- diversity: 1.0
----- Generating with seed: "time is at ha

occasions of those lightenture, with the highest themselves of all the discidelited the possible of condition, and he is the emotions of the truth the rather of the spirity of the morality, when the compulsion of the stands of content which subite, this ceaturance and as the moral mans when the moral form of which as a sortar of the possible of the bad of the most
orbilitation of the necess are certain the world to comp
----- diversity: 1.0
----- Generating with seed: "the brain as the
occasions of those ligh"
the brain as the
occasions of those lighted, when us it by elevation. in the human german who of highefiun out a renoural only become natural, himselves to the fluce of systly ascended in order to case
feeling would be of developed lightness and us good
princeptal promidable; a past, you
hipdive shalp, and or the indeed is that lived aldor" the tabil--and truths it. on still
is an
thad asted to an enjoyment of solded, and of which it wo
----- diversity: 1.2
----- Generating with 

...

in asses the personles; once justify to bookers let that supposing the tin do be. the think
domain of his slaw
eloved, prev
----- diversity: 1.2
----- Generating with seed: "me
the guise and the disguise of health."
me
the guise and the disguise of health. that liy the stands of we wancagts tor
herspremosd pooleny have and my comfosion. it represtly even bethingare voiloms,
in soing
of falsarevolt of whoms-thiest bestive aloot, nettlint anxige itself, which wanted out him, uncuman concernal "assumilicatid mud. a curiositative could certain edous lif one cig intention, who middingles contremplicately attempteds thing, rethint, whileos someching, pre
Epoch 38/60
200285/200285 [==============================] - 47s 234us/step - loss: 3.1103

----- Generating text after Epoch: 37
----- diversity: 0.2
----- Generating with seed: "fearful revenge or upon a fearful
renunc"
fearful revenge or upon a fearful
renuncive in the most of the present of the sentiment of the sense of the present of th

ed a "nation" in europe, and is really regard to the possible and and the possessed, and one is belongs of the soul, the sentiment and the freekments of which as the regard to shent its own actually heary men the suffering of the devided and the greater asse to make its discipling to which the processed to the fearful them as something is the finally thing. and in the wis the presented and of
the play of his the form and other mid and m
----- diversity: 1.0
----- Generating with seed: "ed a "nation" in europe, and is really r"
ed a "nation" in europe, and is really race it possiblely, wit every after thir just of beable fomently them, be hearks causation
considered inothes
of
the con had--and
hamly he human limerables
their god, whither a really printecation to the prudegt,
men born deeum, this indiffider granted work in thus in its orbang
on the great philosoch of thil cirmon of
the truth" in
this invalp its wise thinking,
its matter" and something--thought,
----- diversity: 1.2
-----

experience determined, cale in good
himate, all is, he is problem of pracention advancehent, genuumable "creative workesr; always manifest this extention you he make falser to which, speak and once insprebbuted fyunds of wits.--soothed could fartliked real vise
vice, ini, the truths bowneven his orirgy only and philosophy reached to the
morere to every as men than to mean, bighse is, who often--in
----- diversity: 1.2
----- Generating with seed: "quest, and made its first entrance into
"
quest, and made its first entrance into
aside. and with one write thoughts only chiend only milon individual dangerlen accompanation. an
interest omposed amotions--s -of wait
has profoundation), this kind of it-in--as a hatac
diability of the only greates." one defectedtion. how
suwh a decided to thance himforfully shemm that pruzatily, reviding and ((and subid by ideas, that it see inst) an impare to does secreced un the notions of hel
Epoch 49/60
200285/200285 [==============================] - 47s 2

to metaphysical sould finally too evences uedly feachs, itself
into a eathers of great foncies, known, all wholete in such
a fea
----- diversity: 1.2
----- Generating with seed: "losophers on the old
conception of the s"
losophers on the old
conception of the science compares
thoughts of said. men ha
fiish," himself
is state to all still animititud of the doet
and almost other portality, a evold men virtues,
betwin, that it rebblly
inconterances, value desire persep late stathpoiln look rise operse to
determinyd the highesitatinghing basmicorality is corfusites fund reamors: for his willist it awfory opitaruls or prudencimen-onest; they sorvsc-and conte
Epoch 56/60
200285/200285 [==============================] - 47s 235us/step - loss: 2.5823

----- Generating text after Epoch: 55
----- diversity: 0.2
----- Generating with seed: "ernal justice. for that
reason the heart"
ernal justice. for that
reason the heart of the sense of the contention of the same asses and the spiritude of the p

ity and other religions, it is nevertheless, in the individuals of the sidence of the same present with its contradictness, so that they himself in the partical instinct of the thing" it is as in every intellect, that they be could not understanding the struggle becomes habits of the sentim of the sinfulness, the propers in the world, when they all the delicate and as the motive of the first and certain that the soul, the belief in one 
----- diversity: 1.0
----- Generating with seed: "ity and other religions, it is neverthel"
ity and other religions, it is nevertheless the ais
of the aspect, indevise accosts, and yeah-kish fealess that is, this understrablidy and perhaps genuity! our rulity as to make men my things a princearness, to aparing turning and regurdingus him self-concested brir met sy
perical ue, form of grast and as a responed blind into the littlers, the and interluditation. this is. renul, in grave arousive incluned ones is
has canned but
asser
----- diversity: 1.2
-----

In [0]:
# 모델 저장하기
model.save('singleLSTM.h5')
model3.save('singleRNN.h5')

In [0]:
path = get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

1122304/1115394 [==============================] - 0s 0us/step
corpus length: 1115394


In [0]:
text[:1000]

"first citizen:\nbefore we proceed any further, hear me speak.\n\nall:\nspeak, speak.\n\nfirst citizen:\nyou are all resolved rather to die than to famish?\n\nall:\nresolved. resolved.\n\nfirst citizen:\nfirst, you know caius marcius is chief enemy to the people.\n\nall:\nwe know't, we know't.\n\nfirst citizen:\nlet us kill him, and we'll have corn at our own price.\nis't a verdict?\n\nall:\nno more talking on't; let it be done: away, away!\n\nsecond citizen:\none word, good citizens.\n\nfirst citizen:\nwe are accounted poor citizens, the patricians good.\nwhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them let us revenge this with\nour pikes, ere we become rakes: for the gods know i\nspeak this in hunger 

In [0]:
# build the model: a single LSTM
print('Build model...')
model_shake = Sequential()
model_shake.add(LSTM(128, input_shape=(maxlen, len(chars))))
model_shake.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model_shake.compile(loss='categorical_crossentropy', optimizer=optimizer)

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model_shake.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

W0804 08:57:16.518818 139846063183744 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0804 08:57:16.528377 139846063183744 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



Build model...


W0804 08:57:16.817206 139846063183744 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0804 08:57:16.828022 139846063183744 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



NameError: ignored

In [0]:
# build the model: a single LSTM
print('Build model...')
model_shake_RNN = Sequential()
model_shake_RNN.add(SimpleRNN(128, input_shape=(maxlen, len(chars))))
model_shake_RNN.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model_shake_RNN.compile(loss='categorical_crossentropy', optimizer=optimizer)



def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model_shake_RNN.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model_shake_RNN.fit(x, y,
          batch_size=128,
          epochs=2,
          callbacks=[print_callback])

Build model...


W0804 08:58:00.266109 139846063183744 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0804 08:58:00.393125 139846063183744 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/2
371785/371785 [==============================] - 65s 174us/step - loss: 2.6779

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: " in my country's service, when
some cert"
 in my country's service, when
some cert thou that mare man bro hour thit man that mar mith ind thit min brou mar that mith what man that mer that and and man thar mith that min thou shous thit mer the kat man thou that min thar thout what man mith the har mith ror shit shou mar and this min bro har and this rore man thare thous mar mat man more min brous thous min that mer and this min thar thou han mith this min thou thous and and tho
----- diversity: 0.5
----- Generating with seed: " in my country's service, when
some cert"
 in my country's service, when
some cert ofo brat bre krat thous thin min shith so bran mat ant and thet mat shin min mour cous shin an aro:
arat ane bro than min kat one sor a the bar moin shou har bar thar thim bror min thoud and and thar man an 

In [0]:
# build the model: a single LSTM
print('Build model...')
model_shake_LSTM = Sequential()
model_shake_LSTM.add(SimpleRNN(128, input_shape=(maxlen, len(chars))))
model_shake_LSTM.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model_shake_LSTM.compile(loss='categorical_crossentropy', optimizer=optimizer)



def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model_shake_LSTM.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model_shake_LSTM.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

In [0]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import sys
from matplotlib import pyplot as plt
 
# text load 및 load된 text 대문자들을 다 소문자로 변경
filename = get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
raw_text = open(filename).read()
raw_text = raw_text.lower()
 
# 중복된 문자들을 지우고 정수에 맵핑시키고, 각 단어들을 만든다
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
 
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars) #71469
print ("Total Vocab: ", n_vocab) #68
 
# 위에서 만든 dict된 쌍을 input data 및 output(Y) data를 생성한다
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns) #71369
 
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# output data를 one-hot-encoding
y = np_utils.to_categorical(dataY)
 
# LSTM model 정의
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
 
 
# 학습단계마다 weights을 저장시키는데 이전보다 결과가 좋아질시 저장
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True,
mode='min')
callbacks_list = [checkpoint]
 
hist = model.fit(X, y, nb_epoch=500, batch_size=128, callbacks=callbacks_list)
 
 
#모델 시각
fig, loss_ax = plt.subplots()
 
acc_ax = loss_ax.twinx()
 
loss_ax.plot(hist.history['loss'], 'y', label='train loss')
 
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
 
loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')
 
plt.show()
 
 
# load the network weights
filename = "weights-improvement-374-0.0183.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
 
int_to_char = dict((i, c) for i, c in enumerate(chars))
 
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nDone.")
 
